In [2]:
import lanfactory
import os
import pickle
import tensorflow as tf

In [2]:
#from config import config_dict

ModuleNotFoundError: No module named 'config'

In [3]:
lanfactory.util

AttributeError: module 'lanfactory' has no attribute 'util'

In [4]:
train_config = lanfactory.config.train_config_mlp
network_config = lanfactory.config.network_config_mlp

In [5]:
pickle.dump(train_config, open('test_dump.py', 'wb'))

In [6]:
pickle.load(open('test_dump.py', 'rb'))

{'batch_size': 128,
 'n_epochs': 100,
 'optimizer': 'adam',
 'learning_rate': 0.002,
 'loss': 'huber',
 'metrics': [<tensorflow.python.keras.losses.MeanSquaredError at 0x7f84e6b19c10>,
 'callbacks': ['checkpoint', 'earlystopping', 'reducelr']}

In [2]:
help(lanfactory)

Help on package lanfactory:

NAME
    lanfactory

PACKAGE CONTENTS
    config (package)
    trainers (package)
    utils (package)

VERSION
    0.0.1

FILE
    /users/afengler/data/software/miniconda3/envs/lanfactory/lib/python3.7/site-packages/lanfactory/__init__.py




In [3]:
help(lanfactory.config)

Help on package lanfactory.config in lanfactory:

NAME
    lanfactory.config

PACKAGE CONTENTS
    network_configs

DATA
    network_config_mlp = {'activations': ['tanh', 'tanh', 'linear'], 'call...
    train_config_mlp = {'batch_size': 128, 'callbacks': ['checkpoint', 'ea...

FILE
    /users/afengler/data/software/miniconda3/envs/lanfactory/lib/python3.7/site-packages/lanfactory/config/__init__.py




In [4]:
lanfactory.config.network_config_mlp

{'layer_types': ['dense', 'dense', 'dense'],
 'layer_sizes': [100, 100, 1],
 'activations': ['tanh', 'tanh', 'linear'],
 'loss': ['huber'],
 'callbacks': ['checkpoint', 'earlystopping', 'reducelr']}

In [5]:
lanfactory.config.train_config_mlp

{'batch_size': 128,
 'n_epochs': 100,
 'optimizer': 'adam',
 'learning_rate': 0.002,
 'loss': 'huber',
 'metrics': [<tensorflow.python.keras.metrics.MeanSquaredError at 0x7f63ad593290>,
 'callbacks': ['checkpoint', 'earlystopping', 'reducelr']}

In [6]:
my_list = os.listdir('/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_10000/')
valid_list = ['/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_10000/' + file_ for file_ in my_list if 'ddm' in file_]

In [7]:
my_network_config = lanfactory.config.network_config_mlp
my_network_config['layer_types'] = ['dense', 'dense', 'dense']
my_network_config['layer_sizes'] = [100, 100, 1]
my_network_config['activations'] = ['tanh', 'tanh', 'linear']
my_network_config['loss'] = ['huber']
my_network_config['callbacks'] = ['checkpoint', 'earlystopping', 'reducelr']

In [8]:
my_train_config = lanfactory.config.train_config_mlp
my_train_config['batch_size'] = 50000
my_train_config['n_epochs'] = 10
my_train_config['training_files'] = valid_list
my_train_config['validation_files'] = valid_list[:5]

In [9]:
#batch_size =my_train_config['batch_size']
my_generator_train = lanfactory.trainers.DataGenerator(file_IDs = valid_list,
                                                       batch_size = my_train_config['batch_size'],
                                                       shuffle = True,
                                                       label_prelog_cutoff_low = 1e-7)
my_generator_val = lanfactory.trainers.DataGenerator(file_IDs = valid_list[:5],
                                                     batch_size = my_train_config['batch_size'],
                                                     shuffle = True,
                                                     label_prelog_cutoff_low = 1e-7)

In [10]:
from copy import deepcopy
my_keras_model = lanfactory.trainers.KerasModel(network_config = deepcopy(my_network_config),
                                                input_shape = my_generator_train.input_dim,
                                                save_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/models/')

In [15]:
lanfactory.utils.save_configs(model_id = my_keras_model.model_id,
                              save_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/models',
                              network_config = my_train_config, #my_network_config,
                              train_config = None, #my_train_config,
                              allow_abs_path_folder_generation = True)

Found folder:  /users
Moving on...
Found folder:  /users/afengler
Moving on...
Found folder:  /users/afengler/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/models
Moving on...
Saved network config
Saved train config


In [12]:
my_model_trainer = lanfactory.trainers.ModelTrainerKerasSeq(train_config = deepcopy(my_train_config),
                        data_generator_train = my_generator_train,
                        data_generator_val = my_generator_val,
                        model = my_keras_model,
                        output_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/models/',
                        warm_start = False,
                        allow_abs_path_folder_generation = True)

Found folder:  /users
Moving on...
Found folder:  /users/afengler
Moving on...
Found folder:  /users/afengler/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/models
Moving on...


In [14]:
my_model_trainer.train_model(save_history = True)

Epoch 1/10
500/500 [==============================] - 6s 10ms/step - loss: 0.8160 - MSE: 7.6671 - Huber: 0.8160 - val_loss: 0.2143 - val_MSE: 6.5396 - val_Huber: 0.2143

Epoch 00001: saving model to /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/models/11b7027ef4a211eba053a0423f3e9a4a_ddm_ckpt.h5
Epoch 2/10
500/500 [==============================] - 5s 10ms/step - loss: 0.1614 - MSE: 3.8472 - Huber: 0.1614 - val_loss: 0.1234 - val_MSE: 3.5633 - val_Huber: 0.1234

Epoch 00002: saving model to /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/models/11b7027ef4a211eba053a0423f3e9a4a_ddm_ckpt.h5
Epoch 3/10
500/500 [==============================] - 5s 10ms/step - loss: 0.0952 - MSE: 2.6154 - Huber: 0.0952 - val_loss: 0.0743 - val_MSE: 2.4849 - val_Huber: 0.0743

Epoch 00003: saving model to /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/models/11b7027ef4a211eba053a0423f3e9a4a_ddm_ckpt.h5
Epoch 4/10
500/500 [==============================] - 5

In [15]:
import pickle
data_example = pickle.load(open('/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_10000/' + '/' + \
                 valid_list[0], 'rb'))